# Data Visualization

---

## There are a seemingly infinite number of different tools for data visualization in Python. For today, we're going to focus on Matplotlib and Seaborn. 

> Matplotlib is a standard, Python, 2D plotting library (https://matplotlib.org/) <br> 
> Seaborn is also a Python, data visualization library built atop Matplotlib (https://seaborn.pydata.org/)

## We'll also delve into some work with geographic plotting using geopandas (http://geopandas.org/) and bokeh (https://bokeh.pydata.org/en/latest/index.html). 

---

In [ ]:
# rendering our plots inline (aka, in our Jupyter notebook) and changing the layout a bit

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# installing all of our libraries

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# setting some more styling

sns.set_style("whitegrid")
sns.set(rc={'figure.figsize': (20, 20)})
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks'])

## We'll pick up where we left off last class using our NYPD Crash Data.

## Again, we need to take a moment and convert some of our dtypes:

## Overplotting

## Addressing Overplotting

## Histograms, Density Plots, and Contour Plots

## The hexbin (Hexagonal Bin Plot) creates a 2-d histogram, where the color signals the number of points within a particular area; The gridsize parameter chooses the size of each bin. 

## Density Plots

## Contour Plots

## Combining plots

### We can combine multiple plots using the ax parameter (think of 'ax' as representative of an individual plot). 

## Adding Geographic Boundaries using Bokeh

In [ ]:
# https://docs.bokeh.org/en/latest/

from bokeh.io import output_file, show 
from bokeh.models import *


map_options = GMapOptions(lat=40.7128, lng=-74.0060, map_type="roadmap", zoom=11)

plot = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options,api_key = "")

source = ColumnDataSource(
    data = dict(
        lat=lat_list,
        lon=lon_list,
        date = date_list,
        time = time_list,
        borough = borough_list, 
        vehicle = vehicle_list
    ))

circle = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), BoxZoomTool())

plot.title.text="NYC Accidents"

plot.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@date' ),
        ( 'time',  '@time' ), 
        ( 'borough', '@borough' ), 
        ( 'vehicle', '@vehicle' )
    ],

    formatters={
        'date' : 'datetime', # use 'datetime' formatter for 'date' field
        'time' : 'printf',
        'borough' : 'numeral',
        'vehicle' : 'numeral'
    },

    mode='vline'
))

# output_file("gmap_plot.html")

show(plot)

---

# Example: Analyzing Citibike Station Activity using Pandas

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])

## First, let's fetch our data as we did in week 1:

## Unlike in Week 1, though, we are using a script that runs continuously using a crontab (seen below) so that our database is continually populating with recent data. 

## The .py script is called citibike_cron_script.py and can be found in the Class 7 folder of the course repo. 

> The crontab used is: 
>> */1 * * * * /Users/siegmanA/anaconda3/bin/python $(which python3) ~/Desktop/NYU-Projects-in-Programming-Fall-2019/\(Class\ 7\)\ Data\ Visualization/citibike_cron_script.py >> ~/Desktop/tmp/citiCron.log 2>&1

## Now we want to create a query that gets us the average capacity of a given station in hourly intervals.

---

## Exercise 1: 

### Create a new column in our df called 'percent_full' that tells us how full a bike station is at a given time

In [ ]:
# let's see how 'full' each bike station is at a given time



---

## Examining Time Series per Station

### Let's create a pivot table to examine the time series for individual stations.

## It looks like there's an erroneous entry where we have a last communication time from 1969. Let's get rid of that. 

Let's limit our plot to just two stations:
* Station 3260 at "Mercer St & Bleecker St"
* Station 161 at "LaGuardia Pl & W 3 St"

which are nearby and tend to exhibit similar behavior. Remember that the list of stations is [available as a JSON](https://feeds.citibikenyc.com/stations/stations.json) 

---

## Exercise 2

### Plot a timeseries graph for stations 3260 and 161 only

---

## Finding Bike Stations with Similar Behavior

For our next analysis, we are going to try to find bike stations that have similar behaviors over time. A very simple technique that we can use to find similar time series is to treat the time series as vectors, and compute their correlation. Pandas provides the `corr` function that can be used to calculate the correlation of columns. (If we want to compute the correlation of rows, we can just take the transpose of the dataframe using the `transpose()` function, and compute the correlations there.)

Let's see the similarities of the two stations that we examined above.

For bookkeeping purposes, we are going to drop stations that generate NaN values, as we cannot use such entries for our analysis.

### Clustering Based on Distances

Without explaining too much about clustering, we are going to use a clustering technique and cluster together bike stations that are "nearby" according to our similarity analysis. For this, we need to first convert our similarities to distance.

We are now going to convert our **similarities** into **distance** metrics. Our distance values will be always positive, and bounded between 0 and 1.

* If two stations have correlation 1, they behave identically, and therefore have distance 0, 
* If two stations have correlation -1, they have exactly the oppositite behaviors, and therefore we want to have distance 1 (the max) 

The clustering code is very simple: The code below will create two groups of stations.

We will now take the results of the clustering and associate each of the data points into a cluster.

Let's see how many stations in each cluster

### Visualizing the Time Series Clusters

We will start by assining a color to each cluster, so that we can plot each station-timeline with the cluster color. (We put a long list of colors, so that we can play with the number of clusters in the earlier code, and still get nicely colored results.)

The plot still looks messy. Let's try to plot instead a single line for each cluster. To represent the cluster, we are going to use the _median_ fullness value across all stations that belong to a cluster, for each timestamp. For that, we can again use a pivot table: we define the `communication_time` as one dimension of the table, and `cluster` as the other dimension, and we use the `median` function. 

For that, we first _join_ our original dataframe with the results of the clustering, using the `merge` command, and add an extra column that includes the clusterid for each station. Then we compute the pivot table.

Now, we can plot the medians for the two clusters.

And just for fun and for visual decoration, let's put the two plots together. We are going to fade a lot the individual station time series (by putting the `alpha=0.005`) and we are going to make more prominent the median lines by increasing their linewidths. We will limit our plot to one week's worth of data: